# Package Loading

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Data Loading
data = pd.read_excel('../raw_data/농협양곡 매출데이터(2017년).xlsx')

# 정상만 인덱싱
data = data[data['정정취소구분']=='정상']
data.head()

,사업장,매출일,매출번호,정정취소구분,거래처유형,매출처명,품목명,생산년도,규격명,양곡분류구분,매출수량,환산중량,매출단가,공급금액
0,본사,2017-01-01,1,정상,계통마트,울산유통센터(소매),아라리찰현미10kg,2016,10kg,잡곡,20.0,200.0,17085.0,341700
1,본사,2017-01-01,1,정상,계통마트,울산유통센터(소매),아라리현미10kg,2016,10kg,잡곡,8.0,80.0,17085.0,136680
2,본사,2017-01-01,2,정상,계통마트,(주)농협유통 흑석점,원삼유기농찹쌀4kg,2016,4kg,잡곡,3.0,12.0,25627.0,76881
3,본사,2017-01-01,3,정상,계통마트,(주)농협유통창동유통센터(소매),찰현미1kg,2016,1kg,잡곡,1.0,1.0,3316.0,3316
4,본사,2017-01-01,4,정상,계통마트,(주)농협유통미아점,찹쌀(10kg)지대,2016,10kg,잡곡,500.0,5000.0,24120.0,12060000


In [3]:
data['양곡분류구분'].describe()

count     336894
unique         5
top           잡곡
freq      200619
Name: 양곡분류구분, dtype: object

## 품목명 분류 전처리

In [4]:
# 백미를 담을 리스트
tmp_rice: List[str] = []

# 잡곡을 담을 리스트
tmp_grain: List[str] = [] 

#### 미곡

In [5]:
rice = data[data['양곡분류구분']=='미곡'].reset_index()
rice.reset_index(drop=True, inplace=True)

tmp_rc = list(set(rice['품목명']))

# 찹쌀 및 잡곡 품목 확인 
for num in range(len(tmp_rc)):
    name = tmp_rc[num]    
    
    if '현미' in name:
        tmp_grain.append(name)
        
    else :
        tmp_rice.append(name)

#### 선물세트

In [6]:
gift = data[data['양곡분류구분']=='선물세트'].reset_index()

tmp_gift = list(set(gift['품목명']))

del_list = ['종이박스 22*16*10 (저장품)', '페트병상품(샘플)']

for num in range(len(tmp_gift)):
    
    name = tmp_gift[num]
    
    if ('잡곡' in name) or ('혼합' in name):
        tmp_grain.append(name)

    elif name in del_list:
        continue
    
    else:
        if '한토래' in name:
            print(name)
            tmp_rice.append(name)
         
        else:
            tmp_grain.append(name)      

한토래나물밥 꾸러미 18입 210g*18
한토래 비빔밥 210g
한토래 시래기밥 210g
한토래 송화버섯밥 210g
한토래우리쌀(진공)2kg
한토래 톳밥 210g
한토래햅쌀세트(PET)
한토래 취밥 210g
한토래 나물밥 꾸러미 12입 210g*12
한토래 나물밥 꾸러미 6입 210g*6
한토래 선물세트(고급형)
한토래 곤드레밥 210g


### PB상품

In [7]:
# pb상품 중 쌀 포함된 품목 찾기
pb = data[data['양곡분류구분']=='PB상품'].reset_index()

tmp_pb = list(set(pb['품목명']))

# 오류치 리스트 
del_list = ['페트박스(1L*3)', '비닐봉투(중)', '사각그립페트병5.3L', '라벨스티커(PET용)', 'PET 1+1 고리(38+38mm)']
rice_list = ['한토래고시히카리2kg(PET)/국산', '한토래PET트레이2KG(2X4)', '한토래PET세트(450g*2)',
             '한토래 이천밥맛좋은쌀2kg(PET)/국산', '농협쌀(진공)500g', '갯반닥서해일미 20kg']

for num in range(len(tmp_pb)):
    
    name = tmp_pb[num]
    
    if ('현미' in name) or ('흑미' in name) or ('잡곡' in name):
        tmp_grain.append(name)
    
    elif name in del_list:
        continue
    
    elif (name in rice_list) or ('찹쌀' in name):
        tmp_rice.append(name)
        
    else:
        tmp_grain.append(name)
         

### 잡곡

In [8]:
grain = data[data['양곡분류구분']=='잡곡']
tmp_gr = list(set(grain['품목명']))

# 쌀 리스트
tmp_list = [
    '황토랑5Kg', '황토랑500g', '황토랑20Kg', '황토랑1Kg', '황토랑10Kg', '황토랑10kg', '황토랑(향기)20Kg', '황토랑(행사)20Kg',
    '황토랑(행사)20kg', '황토랑(쿠욱)20kg', '황토랑(갯벌)20kg', '황토랑(갯벌)10kg', '향기로전하는쌀7kg', '향기로전하는쌀7Kg',
    '향기로전하는쌀20Kg', '향기로전하는쌀 3.5kg', '향기로전하는쌀 3.5Kg', '함열청결미20kg', '함열청결미 20kg', '함열청결미 10kg',
    '한토래우리쌀(진공)2kg(10)', '한토래 20kg', '한눈에반한쌀(신)5kg', '하얀미소20K(2018)', '하얀미소20K(2017)', '토실미/2018년산/20kg',
    '토실미/2018년산/10kg', '토실미/2017년산/20kg', '토실미/2017년산/10kg', '토박이20kg', '탑마루쌀 20kg', '탑마루 10kg', '쿠~욱20Kg',
    '친환경양반쌀/2017년산/10kg', '추청쌀20kg', '추청쌀10kg', '철원오대쌀/1kg', '천지향2kg', '찰벼/40Kg', '찰벼(찰벼(일반))/1Kg/PP대/특(1등)/무농약찰벼/무농약',
    '찰벼(찰벼(일반))/1Kg/1등급', '찰벼(찰벼(일반))/1Kg', '찰벼(찰벼(기타))/1Kg/PP대/유기농산물/복수이상상품', '찰벼(찰벼(기타))/1Kg/PP대/무농약/복수이상상품',
    '진천추청쌀20kg(2018)', '진천추청쌀20kg(2017)', '진천추청쌀20kg','진천추청쌀10kg(2018)','진천추청쌀10kg(2017)','진천추청쌀10kg','진천쌀4kg(2018)',
    '진천쌀4kg(2017)','진천쌀2kg(2018)','진천쌀2kg(2017)','진천쌀20kg(2018)','진천쌀20kg(2017)','진천쌀10Kg(2018)','진천쌀10Kg(2017)',
    '자연담은쌀20kg(2018)', '자연담은쌀20kg(2017)','익산탑마루쌀20kg','이천쌀/10kg','유기농쌀10kg','우리집밥맛쌀/2018년산/20kg','우리집밥맛쌀/2017년산/20kg',
    '우리강산애 햅쌀3.2kg', '우렁이랑쌀20Kg','우렁이랑쌀20kg','우렁이랑쌀10Kg','우렁이랑5Kg','우렁이랑500g','우렁이랑20Kg(유기농)','우렁이랑1Kg',
    '우렁이랑10Kg(유기농)', '우렁이랑10Kg','용의눈 20kg','용의눈 10Kg','예살비20kg','예살비 5kg','예살비 4kg','연이랑20Kg','연이랑10Kg',
    '연이랑10kg','여심 20kg', '양반쌀/2019년산/2kg','양반쌀/2018년산/5kg','양반쌀/2018년산/4kg','양반쌀/2018년산/3kg','양반쌀/2018년산/2kg*2set',
    '양반쌀/2018년산/20kg', '양반쌀/2018년산/1kg','양반쌀/2018년산/10kg','양반쌀/2017년산/5kg','양반쌀/2017년산/4kg','양반쌀/2017년산/3kg','양반쌀/2017년산/20kg',
    '양반쌀/2017년산/1kg', '양반쌀/2017년산/10kg','안동양반쌀/2018년산/20kg','안동양반쌀/2018년산/10kg','안동양반쌀/2017년산/20kg',
    '안동양반쌀/2017년산/10kg', '안동미인쌀/2018년산/10kg','안동미인쌀/2017년산/15kg','안동미인쌀/2017년산/10kg','아침햇쌀10kg(2018)',
    '아침햇쌀 4kg(2018)', '쌀의신동20Kg','쌀의신동1kg','생거진천쌀4kg(선물셋트)2018','생거진천쌀4kg(선물셋트)2017/GAP','생거진천쌀2kg(2018)',
    '생거진천쌀2kg(2017)/GAP', '생거진천쌀20kg/GAP','생거진천쌀20kg(2018)','생거진천쌀20kg(2017)/GAP','생거진천쌀10kg/GAP','생거진천쌀10kg(2018)',
    '생거진천쌀10kg(2017)/GAP', '벼(일반계(논벼))/1Kg/포/친환경(무농약)/해안면','벼(일반계(논벼))/1Kg/PE대/원료/일반/국산','벼(논벼)(백옥찰)/1Kg',
    '벼(논벼)(기타)/1Kg/PP대/유기농산물/복수이상상품', '밭벼(밭벼(일반))/1Kg/지대/평택시(고덕면)','누네찬쌀20kg(2018)','누네찬쌀20kg(2017)',
    '누네찬쌀20kg(2016)', '누네찬쌀10kg(2018)','누네찬쌀10kg(2017)','농협쌀(진공)500g','내고향황토쌀20kg','내고향황토쌀20Kg','내고향황토쌀10kg',
    '내고향황토쌀10Kg', '갯벌이랑20Kg','갯벌이랑10Kg','간척지5Kg','간척지5kg','간척지20Kg','간척지10Kg','간척지10kg', '아침햇쌀(공매곡)(2017년)20kg', '한토래우리쌀(진공)2kg',
    '청결미20kg(2018년산)판매품', '신동진쌀20kg(간척지4)','갯벌이랑20kg','쌀(백미)/20Kg/포/소나무(간척지2)', '쌀(백미)/20Kg/포/행복내린쌀(간척지3)',
    '쌀(백미)/2018년산/1Kg', '친환경양반쌀/2018년산/10kg', '하얀미소20K(2019)', '풍광수토20Kg', '풍광수토20kg', '풍광수토10kg', '토실미/2019년산/20kg',
    '토실미/2019년산/10kg', '탑마루쌀 5kg', '쿠욱20Kg', '자연담은쌀20kg(간척지5)', '우렁이랑10kg(유기농)', '오대쌀1kg', '예살비 1kg', '찰벼(찰벼(기타))/1Kg/1등급/국산',
    '여심신동진 20kg', '여심신동진 10kg' ,'양반쌀/2019년산/4kg', '쌀의신동진10kg', '쌀(일반미)/10Kg/포', '쌀(일반미)/10Kg/군산시', '쌀', '한토래 신동진20kg',
    '보석닮은쌀 20kg', '땅기운쌀 5kg', '땅기운쌀 10kg', '고마운땅농협쌀 10kg', '간척지20Kg(2017년산)', '무안풍광수토 20kg', '누네찬쌀10kg(2019)', '내린천일반미 2kg',
    '밥쌀용일품1kg(2018년산)', '생거진천쌀10kg(2019)', '새청무쌀1kg', '생거진천쌀4kg(비닐)2019', '한토래 햅쌀세트(PET)']

for num in range(len(tmp_gr)):
    
    name = tmp_gr[num]

    if name in tmp_list:
        tmp_rice.append(name)
        
    elif ('현미' in name) or ('흑미' in name) or ('잡곡' in name) or ('찹쌀보리' in name):
        tmp_grain.append(name)

    elif '찹쌀' in name:
        tmp_rice.append(name)
        
    elif ('왕겨' in name) or ('청미' in name) or ('싸라기' in name):
        continue
        
    else:
        tmp_grain.append(name)

### 해당없음

In [9]:
nn = data[data['양곡분류구분']=='해당없음']
tmp_nn = list(set(nn['품목명']))

tmp_list = ['농협청결미 20kg', '향기로전하는쌀1.75Kg', '향기로전하는쌀 1.75kg', '한토래450g (2개입)PET상품', '하늘쌀1Kg',
            '추청쌀950g(PET)', '천년의 숨결 20kg', '찰보리/1Kg', '이천푸드20kg', '예살비 10kg', '양반쌀/2018년산/40kg',
            '양반쌀/2017년산/500g', '양반쌀/2017년산/40kg', '안동미인쌀/2018년산/20kg', '안동미인쌀/2017년산/20kg', '아침햇쌀20kg(2018)',
            '신동진백미/1kg', '생거진천쌀500g/GAP', '생거진천쌀500g(2018)', '벼(논벼)혼합/2017년산/1Kg', '벼(논벼)(기타)/1Kg(몽탄)',
            '벼(논벼)(기타) 신동진', '미곡종자(기타)/1Kg(일미종자)', '무농약신동진 5kg', '메뚜기황토쌀/2018년산/20kg', '메뚜기황토쌀/2017년산/20kg',
            '기타미곡류(기타)/1Kg(신동진종자)', 'NH신동진쌀10kg', 'NH신동진쌀 20kg', 'NH신동진쌀 10kg', 'NH신동진 4kg', 'NH신동진 20kg', '17년산 일반벼',
            '양반쌀/2018년산/500g', '아침햇쌀20kg(2019)', '안동미인쌀/2019년산/20kg', 'NH신동진 500g', '18년산 일반벼','양반쌀30kg',
            '벼(판매품)/1Kg 2019년산', '무농약신동진 10kg', '왕곡새로미20kg', '벼(논벼)19년산조벼', '무농약신동진 20kg', '신동진쌀20kg(간척지4)',
            '나비랑여문쌀20kg', '햇쌀한공기(포장재)', '벼(논벼)(판매품)/1Kg 2018년산', '풍광수토10kg', '한토래450g * 2개입 미니PET세트(20)',
            '보석닮은쌀 20kg', '신동진쌀20kg', '2018년산보람찬벼', '벼(논벼)일미/1Kg/무안군', '수원5291kg(2018년산/판매품)', '메뚜기황토쌀/2019년산/20kg',
            '이천푸드', 'NH신동진쌀 1kg', '신동진 백미1kg', '16년산 일반벼', '15년산주정용쌀', '강과강사이 10kg', '아침햇쌀20kg']

del_list = ['청치(1kg)', '쇄미1kg', '카본테이프 (저장품)', '비닐봉투10 (저장품)', '비닐봉투21 (저장품)', '종이박스 (저장품)', '비닐봉투 7 (저장품)', '띠지 (저장품)',
            '비닐봉투 6 (저장품)', '비닐봉투 4 (저장품)', '떡국떡 비닐봉투', '비닐봉투 9 (저장품)', '저울용라벨지 (저장품)', '박스테이프 (저장품)']

for num in range(len(tmp_nn)):
    
    name = tmp_nn[num]
    
    if name in tmp_list:
        tmp_rice.append(name)
        
    elif ('현미' in name) or ('흑미' in name) or ('잡곡' in name) or ('찹쌀보리' in name):
        tmp_grain.append(name)
    
    elif ('왕겨' in name) or (name in del_list):      
        continue
        
    else:
        tmp_grain.append(name)
        print(name)

수수(차수수)/1Kg
미강(1kg)
건강가득담은2종세트3kg
들깨(16년산)/고려중앙학원/1Kg
참깨(흰깨)/고려중앙학원/1Kg
미강1kg
보리쌀(늘보리쌀)
콩(백태)/100g


In [10]:
tmp_rice = list(set(tmp_rice))
len(tmp_rice)

1794

In [11]:
tmp_grain = list(set(tmp_grain))
len(tmp_grain)

2801

In [12]:
data_copy = data.copy()
data_copy.loc[data_copy['품목명'].isin(tmp_rice), '품종'] = '미곡'
data_copy.loc[data_copy['품목명'].isin(tmp_grain), '품종'] = '잡곡'
data_copy

,사업장,매출일,매출번호,정정취소구분,거래처유형,매출처명,품목명,생산년도,규격명,양곡분류구분,매출수량,환산중량,매출단가,공급금액,품종
0,본사,2017-01-01,1,정상,계통마트,울산유통센터(소매),아라리찰현미10kg,2016,10kg,잡곡,20.0,200.00,17085.0,341700,잡곡
1,본사,2017-01-01,1,정상,계통마트,울산유통센터(소매),아라리현미10kg,2016,10kg,잡곡,8.0,80.00,17085.0,136680,잡곡
2,본사,2017-01-01,2,정상,계통마트,(주)농협유통 흑석점,원삼유기농찹쌀4kg,2016,4kg,잡곡,3.0,12.00,25627.0,76881,미곡
3,본사,2017-01-01,3,정상,계통마트,(주)농협유통창동유통센터(소매),찰현미1kg,2016,1kg,잡곡,1.0,1.00,3316.0,3316,잡곡
4,본사,2017-01-01,4,정상,계통마트,(주)농협유통미아점,찹쌀(10kg)지대,2016,10kg,잡곡,500.0,5000.00,24120.0,12060000,미곡
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336889,안성지사,2017-12-26,117,정상,기타,농업회사법인유한회사광복,기장/1Kg/PP대/매취사업,2017,1Kg,잡곡,20000.0,20000.00,4420.0,88400000,잡곡
336890,안성지사,2017-12-28,3,정상,기타,서래정,한토래 햅쌀세트(PET),2017,4kg,잡곡,10.0,40.00,20500.0,205000,미곡
336891,안성지사,2017-12-28,3,정상,기타,서래정,한토래 햅쌀잡곡3종세트(PET),2016,3.2Kg,해당없음,3.0,9.60,25000.0,75000,잡곡
336892,안성지사,2017-12-28,3,정상,기타,서래정,한토래 햅쌀잡곡2종세트(PET),2016,2850g,해당없음,7.0,19.95,25000.0,175000,잡곡


In [13]:
data_copy[data_copy['품종'].isnull()].groupby(['품목명']).sum()

,매출번호,생산년도,매출수량,환산중량,매출단가,공급금액
품목명,,,,,,
PET 1+1 고리(38+38mm),150072,0,7001.0,7001.0,135.00,315045
떡국떡 비닐봉투,150161,0,13354.0,13354.0,348.00,2158410
띠지 (저장품),150096,0,23623.0,23623.0,254.00,1958814
라벨스티커(PET용),500548,0,408581.0,408581.0,1141.00,41740717
박스테이프 (저장품),150047,0,59.0,59.0,29473.00,579953
비닐봉투 4 (저장품),100078,0,30335.0,30335.0,9411.00,2405718
비닐봉투 6 (저장품),150087,0,39834.0,39834.0,5960.00,3272102
비닐봉투 7 (저장품),150168,0,170635.0,170635.0,267.00,16516408
비닐봉투 9 (저장품),100107,0,46208.0,46208.0,23197.00,10188517


In [14]:
data_copy.dropna(inplace=True)
data_copy.reset_index(drop=True, inplace=True)
data_copy

,사업장,매출일,매출번호,정정취소구분,거래처유형,매출처명,품목명,생산년도,규격명,양곡분류구분,매출수량,환산중량,매출단가,공급금액,품종
0,본사,2017-01-01,1,정상,계통마트,울산유통센터(소매),아라리찰현미10kg,2016,10kg,잡곡,20.0,200.00,17085.0,341700,잡곡
1,본사,2017-01-01,1,정상,계통마트,울산유통센터(소매),아라리현미10kg,2016,10kg,잡곡,8.0,80.00,17085.0,136680,잡곡
2,본사,2017-01-01,2,정상,계통마트,(주)농협유통 흑석점,원삼유기농찹쌀4kg,2016,4kg,잡곡,3.0,12.00,25627.0,76881,미곡
3,본사,2017-01-01,3,정상,계통마트,(주)농협유통창동유통센터(소매),찰현미1kg,2016,1kg,잡곡,1.0,1.00,3316.0,3316,잡곡
4,본사,2017-01-01,4,정상,계통마트,(주)농협유통미아점,찹쌀(10kg)지대,2016,10kg,잡곡,500.0,5000.00,24120.0,12060000,미곡
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336889,안성지사,2017-12-26,117,정상,기타,농업회사법인유한회사광복,기장/1Kg/PP대/매취사업,2017,1Kg,잡곡,20000.0,20000.00,4420.0,88400000,잡곡
336890,안성지사,2017-12-28,3,정상,기타,서래정,한토래 햅쌀세트(PET),2017,4kg,잡곡,10.0,40.00,20500.0,205000,미곡
336891,안성지사,2017-12-28,3,정상,기타,서래정,한토래 햅쌀잡곡3종세트(PET),2016,3.2Kg,해당없음,3.0,9.60,25000.0,75000,잡곡
336892,안성지사,2017-12-28,3,정상,기타,서래정,한토래 햅쌀잡곡2종세트(PET),2016,2850g,해당없음,7.0,19.95,25000.0,175000,잡곡


# csv로 저장

In [15]:
data_copy.to_csv('./preprocessed_data/preprocessed_data_2017.csv', index=False)